# **<span style="color:black;">US Dataset: Loading and Cleaning</span>**

## **Purpose of Notebook**
Preparing the raw data for EDA, merging datasets, clearning, dropping columns, filling blanks with 'missing', fixing formats etc

## **Problem Statement and Sourcing the Dataset**
Datasets are from:  https://www.nhtsa.gov/laws-regulations/standing-general-order-crash-reporting
Including three different datasets
* ADAS = Advanced Driver Assistance Systems
* ADS = Automated Driving Systems
* OTHER = Misclassed/No Available Information


This includes fully autonomous and driver assisted accidents, other includes any other accidents that are not linked as ADS/ADAS.

Goal combine these datasets but only the unique rows, hence finding a primary unique key to identify the duplicates and overlapped rows (as source dataset entity acknowledges this)

Additionally finding missing data, rows which are important, fixing formatting, renaming rows etc ready for Exploratory Data Analysis

---

## **Contents of notebook**
### **Section 1: Loading and Merging Dataset**
```python 
- [x] Loading the datasets
- [x] Getting Basic Information
- [x] Checking How Much missing data before merging
- [x] Finding Primary Keys and Merging Dataset
- [x] Reduce the report versions to single 
```
### **Section 2: Dropping Unneeded Columns**
```python 
- [x] Combine connected columns
- [x] Drop unneeded columns
```
### **Section 3: Handling missingness**
```python 
- [x] Checking unknown vs missing data
- [x] Which columns have the most missing data
- [x] Unknown and missing combined
```
### **Section 4: Fixing Formats and Standardising**
```python 
- [x] Convert Datetime
- [x] Convert Numerical and String Columns
- [x] Renaming Columns and Values to match STATS19
- [x] Finding Primary Keys and Merging Dataset
```
### **Section 5: Save to File**
---

**Import Necessary Libraries and Modules**

In [1]:
import pandas as pd # For data manipulation
import numpy as np # For numerical operations
import itertools # For creating combinations when finding primary key
from tabulate import tabulate # For pretty printing 
from dateutil import parser


## **<span style="color:black;">Section 1: Loading and Merging Datasets</span>**


**Loading the datasets**
- Change the path when reusing this notebook

In [2]:
# Loading the datasets (change the relative paths as needed)
df_adas = pd.read_csv('/Users/mahnooriqbal/COMP702 Project/ML-AI-Risk-Analysis-AV-Data-/Datasets/US/SGO-2021-01_Incident_Reports_ADAS.csv')
df_ads = pd.read_csv('/Users/mahnooriqbal/COMP702 Project/ML-AI-Risk-Analysis-AV-Data-/Datasets/US/SGO-2021-01_Incident_Reports_ADS.csv')
df_other = pd.read_csv('/Users/mahnooriqbal/COMP702 Project/ML-AI-Risk-Analysis-AV-Data-/Datasets/US/SGO-2021-01_Incident_Reports_OTHER.csv')

**Finding the Primary Keys**
- Extremely important for merging to ensure theres no duplicates 
- Otherwise we need to add

In [3]:
def find_primary_keys_common(df, common_columns):
    """
       Purpose: Finding the primary keys in each dataframe using only common columns
       Methods:
         - Check each row for this column is unique (count of column vals = number of rows)
         - If not, then use itertools to experiment with every combination to find what combination is unique
       Input: Dataframe, Set of common columns
       Output: Keys within a string for easy printing
    """
    # Check for single primary key
    for col in common_columns:
        if df[col].is_unique:
            return f"Primary key: ({col})"
    
    # Loop to check every combination of the common columns
    for i in range(2, len(common_columns) + 1):  # keep increasing if no primary key found for double, triple, etc.
        for combo in itertools.combinations(common_columns, i):
            if df[list(combo)].drop_duplicates().shape[0] == df.shape[0]:  # check if count = total rows in df
                return f"Primary key: {combo}"
            
    return "No primary key found"  # just in case there is no primary key

# Find common columns across all three datasets
common_columns = set(df_adas.columns).intersection(set(df_ads.columns)).intersection(set(df_other.columns))

# Calling Function
print("ADAS: ", find_primary_keys_common(df_adas, common_columns))
print("ADS: ", find_primary_keys_common(df_ads, common_columns))
print("OTHER: ", find_primary_keys_common(df_other, common_columns))

ADAS:  Primary key: ('Report ID', 'Report Version')
ADS:  Primary key: ('Report ID', 'Report Version')
OTHER:  Primary key: ('Report Type', 'Report ID')


**General Basic Information**

In [4]:
# Basic Information about the datasets
keys = ['Report ID','Report Version'] # works for all 
data = {
    "Dataset": ["ADAS",
                "ADS",
                "OTHER"],

    "Shape": [df_adas.shape,
              df_ads.shape,
              df_other.shape],

    f"{keys}": [ # ensure primary key is unique
    df_adas[keys].drop_duplicates().shape[0],
    df_ads[keys].drop_duplicates().shape[0],
    df_other[keys].drop_duplicates().shape[0]],

    "Duplicates": [df_adas.duplicated(subset=keys).sum(),
                   df_ads.duplicated(subset=keys).sum(),
                   df_other.duplicated(subset=keys).sum()]}
df_info = pd.DataFrame(data)
print(tabulate(df_info, headers='keys', tablefmt='pretty')) # simple table to easily see

+---+---------+-------------+---------------------------------+------------+
|   | Dataset |    Shape    | ['Report ID', 'Report Version'] | Duplicates |
+---+---------+-------------+---------------------------------+------------+
| 0 |  ADAS   | (3905, 137) |              3905               |     0      |
| 1 |   ADS   | (2173, 137) |              2173               |     0      |
| 2 |  OTHER  | (3547, 137) |              3547               |     0      |
+---+---------+-------------+---------------------------------+------------+


**Checking how much missing dataset**
- Dataset 3 can be dropped firstly because 'other' is not adas, ads or conventional hence it wont dervive as much useful information for the project aim 
- Highly missing hence also easier to drop

In [5]:
# Missing values percentage overall per dataset
def missing_percentage(df):
        """
        Purpose: Calculate the percentage of missing values in a DataFrame.
        Input: DataFrame
        Output: Percentage of missing values in the DataFrame
        """
        total_cells = df.size
        total_missing = df.isnull().sum().sum()
        return (total_missing / total_cells) * 100

print("=== Missing Values Percentage ===")
print(f"Dataset 1 missing: {missing_percentage(df_adas):.2f}%")
print(f"Dataset 2 missing: {missing_percentage(df_ads):.2f}%")
print(f"Dataset 3 missing: {missing_percentage(df_other):.2f}%")

=== Missing Values Percentage ===
Dataset 1 missing: 13.04%
Dataset 2 missing: 9.84%
Dataset 3 missing: 41.06%


**How much missingness per column**
- Columns which are extremely highly missing must be dropped as not enough raw information for imputation

In [6]:
missing_percentage = (df_other.isnull().mean() * 100).sort_values(ascending=False)
print("\nMissing Values Percentage by Column:")
print(missing_percentage.to_string(float_format="%.2f%%"))


Missing Values Percentage by Column:
State or Local Permit             100.00%
Other Federal Reg. Exemption      100.00%
Federal Regulatory Exemption?     100.00%
Weather - Other Text              100.00%
ADAS/ADS Hardware Version          99.94%
Investigating Officer Email        99.94%
Investigating Officer Phone        99.94%
State or Local Permit?             99.94%
Serial Number                      99.94%
ADAS/ADS Software Version          99.94%
Other Reporting Entities?          99.92%
Narrative - CBI?                   99.89%
Source - Other Text                99.69%
ADAS/ADS System Version            99.52%
Latitude                           99.46%
Longitude                          99.46%
Investigating Officer Name         99.41%
Investigating Agency               99.32%
ADS Equipped?                      98.51%
Zip Code                           98.25%
Posted Speed Limit (MPH)           98.20%
Automation System Engaged?         98.20%
Mileage                            98.

**Ensuring the columns fully match for inner join**

In [7]:
# Find overlapping columns between df_adas and df_ads
print(f"Shape of df_adas: {df_adas.shape}")
print(f"Shape of df_ads: {df_ads.shape}")

# Ensuring the columns are the same so merge is easy
overlapping_columns = set(df_adas.columns).intersection(set(df_ads.columns))
print(f"Overlapping columns ({len(overlapping_columns)}):")

Shape of df_adas: (3905, 137)
Shape of df_ads: (2173, 137)
Overlapping columns (137):


**Add Extra Column to state which dataset it is from**

In [8]:
# Add a source column
df_adas['Source'] = 'ADAS'
df_ads['Source'] = 'ADS'

# combining the two dataframes but using concat instead of merge to ensure 274 columns are kept
merged_df = pd.concat([df_adas, df_ads], ignore_index=True)

# Checking the result of the merge
print(f"Shape of merged dataframe: {merged_df.shape}")
print(tabulate(merged_df.head(10), headers='keys', tablefmt='grid', showindex=True))

Shape of merged dataframe: (6078, 138)
+----+-------------+------------------+------------------------------------------+---------------+----------------+---------------+--------------------------+-------------+-----------------+---------------------------------------------------+---------------+----------------+-------------------+--------------+------------------------+-------------------+-----------+---------------------+--------------------------------+---------------------------------------------------------------+---------------------------------+-------------------------------+-----------------------------------------------------------+-----------------------------------+---------------------------------+-----------------------------------------------------------+-----------------------------------+---------------------------------+-----------------------------+-----------------------------------+----------------------------------+---------------------------------+--------------

**Reducing the Multiple Report Versions**
- Whilst report ID is a primary key there are multiple version of report id as more information develops another report is added 
- Latest report is saved and rest can be removed (going from 6078 to 4372)

In [9]:
# For each report ID count how many versions there are
version_counts = merged_df.groupby('Report ID')['Report Version'].nunique()
multi_version = version_counts[version_counts > 1]
print(f"\nReport IDs with multiple versions (count: {len(multi_version)}):")
multi_version = multi_version.sort_values(ascending=False)
print(multi_version)

# Dropping all the old versions of the reports
merged_df = merged_df.loc[merged_df.groupby('Report ID')['Report Version'].idxmax()].reset_index(drop=True)

# Final Merged DataFrame with latest versions
print(f"\nShape of merged dataframe after dropping old versions: {merged_df.shape}")
print("\nFirst 10 Rows of latest versions:")
print(tabulate(merged_df.head(10), headers='keys', tablefmt='grid', showindex=True))


Report IDs with multiple versions (count: 1497):
Report ID
945-8258      9
28349-4648    8
855-1341      5
1243-1379     5
753-4134      5
             ..
13781-3716    2
13781-3715    2
13781-3714    2
13781-3713    2
988-3684      2
Name: Report Version, Length: 1497, dtype: int64

Shape of merged dataframe after dropping old versions: (4372, 138)

First 10 Rows of latest versions:
+----+-------------+------------------+---------------------------+---------------+----------------+---------------+--------------------------+-------------+-----------------+-----------------+---------------+-------------+-------------------+--------------+------------------------+-------------------+-----------+---------------------+--------------------------------+-----------------------------------------------------------+---------------------------------+-------------------------------+----------------------------------------------------------------------+-----------------------------------+---------

## **Section 2: Dropping Unneccessary Columns**

**Listing all the columns to check if any interconnected**

In [10]:
# Listing all the columns of the merged DataFrame vertically
for column in merged_df.columns:
    print(column)

Report ID
Report Version
Reporting Entity
Report Type
Report Month
Report Year
Report Submission Date
VIN
VIN - Unknown
Serial Number
Make
Model
Model - Unknown
Model Year
Model Year - Unknown
Same Vehicle ID
Mileage
Mileage - Unknown
Driver / Operator Type
ADAS/ADS System Version
ADAS/ADS System Version - Unk
ADAS/ADS System Version CBI
ADAS/ADS Hardware Version
ADAS/ADS Hardware Version - Unk
ADAS/ADS Hardware Version CBI
ADAS/ADS Software Version
ADAS/ADS Software Version - Unk
ADAS/ADS Software Version CBI
Other Reporting Entities?
Other Reporting Entities? - Unk
Other Reporting Entities? - NA
Federal Regulatory Exemption?
Other Federal Reg. Exemption
Federal Reg. Exemption - Unk
Federal Reg. Exemption - No
State or Local Permit?
State or Local Permit
ADS Equipped?
Automation System Engaged?
Operating Entity
Operating Entity - Unknown
Source - Complaint/Claim
Source - Telematics
Source - Law Enforcement
Source - Field Report
Source - Testing
Source - Media
Source - Other
Source - O

### **List of Connected Columns**

**Double Columns** = If value for column is empty then unknown column has 'Y' value 
1. VIN, VIN - Unknown
2. Model, Model - Unknown
3. Model Year, Model Year - Unknown
4. Mileage, Mileage - Unknown
5. Incident Time (24:00), Incident Time - Unknown
6. Latitude, Latitude - Unknown
7. Longitude, Longitude - Unknown
8. Address, Address - Unknown
9. City, City - Unknown
10. Zip Code, Zip Code - Unknown
11. Operating Entity, Operating Entity - Unknown
12. Investigating Agency, Investigating Agency - Unknown
13. Investigating Officer Name, Inv. Officer Name - Unknown
14. Investigating Officer Phone, Inv. Officer Phone - Unknown
15. Investigating Officer Email, Inv. Officer Email - Unknown
16. Posted Speed Limit (MPH), Posted Speed Limit - Unknown
17. SV Precrash Speed (MPH), SV Pre-crash Speed - Unknown

**Multiple Value Columns** = Useful data split into multiple rows which can be combined
1. ADAS/ADS System Version, ADAS/ADS System Version - Unk, ADAS/ADS System Version CBI
2. ADAS/ADS Hardware Version, ADAS/ADS Hardware Version - Unk, ADAS/ADS Hardware Version CBI
3. ADAS/ADS Software Version, ADAS/ADS Software Version - Unk, ADAS/ADS Software Version CBI
4. Other Reporting Entities?, Other Reporting Entities? - Unk, Other Reporting Entities? - NA
5. Federal Regulatory Exemption?, Other Federal Reg. Exemption, Federal Reg. Exemption - Unk, Federal Reg. Exemption - No
6. State or Local Permit?, State or Local Permit
7. Source - Complaint/Claim, Source - Telematics, Source - Law Enforcement, Source - Field Report, Source - Testing, Source - Media, Source - Other, Source - Other Text
8. Weather - Clear, Weather - Snow, Weather - Cloudy, Weather - Fog/Smoke, Weather - Rain, Weather - Severe Wind, Weather - Other, Weather - Other Text
9. CP Contact Area - Rear Left, CP Contact Area - Left, CP Contact Area - Front Left, CP Contact Area - Rear, CP Contact Area - Top, CP Contact Area - Front, CP Contact Area - Rear Right, CP Contact Area - Right, CP Contact Area - Front Right, CP Contact Area - Bottom
10. SV Contact Area - Rear Left, SV Contact Area - Left, SV Contact Area - Front Left, SV Contact Area - Rear, SV Contact Area - Top, SV Contact Area - Front, SV Contact Area - Rear Right, SV Contact Area - Right, SV Contact Area - Front Right, SV Contact Area - Bottom
11. Data Availability - EDR, Data Availability - Police Rpt, Data Availability - Telematics, Data Availability - Complaints, Data Availability - Video, Data Availability - Other, Data Availability - No Data
12. Within ODD?, Within ODD? - CBI
13. Narrative, Narrative - CBI?

**Dropping the 'Unknown' double Columns**
- For the double columns, if a value is missing then the unknown version of the column has a Y value within it 
- This column can easily be dropped as the main column has a NaN which can be imputted or listed as 'missing' later on reducing the number of rows significantly

**Combining the values in multivalue columns**
- List all the columns which are connected and then merging whatever values are in those into a main column
- eg: CP Contact area
Example Row Data
  - 'CP Contact Area - Rear Left' 
  - 'CP Contact Area - Left'
  - 'CP Contact Area - Front Left'
  - 'CP Contact Area - Rear'
  - 'CP Contact Area - Top'  | **Y**
  - 'CP Contact Area - Front'
  - 'CP Contact Area - Rear Right'
  - 'CP Contact Area - Right'
  - 'CP Contact Area - Front Right'
  - 'CP Contact Area - Bottom'

Turns into: CP Contact area -> CP Contact Area - Top

In [11]:
# drop all the unknown columns as they can be empty and imputed later (22 columns)
merged_df = merged_df.loc[:, ~merged_df.columns.str.contains('Unknown')]
print(tabulate(merged_df.head(2), headers='keys', tablefmt='grid'))
print(f"Number of columns after dropping 'Unknown': {merged_df.shape[1]}")
# Combine columns with 'Y' values into a single column with their col name
def combine_and_drop(merged_df, new_col_name, cols_to_combine):
    '''
        Purpose: Combine column values into 1, if they contain Y then place the column name 
        Input:
            merged_df: DataFrame to modify
            new_col_name: Name of the new column to create
            cols_to_combine: List of columns to combine
        Output: DataFrame with the new column and specified columns dropped
    '''
    def combine_values(row):
        """
        Purpose: Combine values from specified columns into a single string if they contain 'Y'.
        Input: Row of DataFrame
        Output: Combined string of column names where the value is 'Y'
        """
        combined_values = []
        for col in cols_to_combine: # go through all columns in list
            if str(row[col]).strip().upper() == 'Y':  # if any value in that row then
                combined_values.append(col.split(' - ')[-1]) # only place what is after the column name 
        return ', '.join(combined_values)

    merged_df[new_col_name] = merged_df[cols_to_combine].apply(combine_values, axis=1)
    merged_df.drop(columns=cols_to_combine, inplace=True)

+----+-------------+------------------+--------------------+---------------+----------------+---------------+--------------------------+-------------+-----------------+---------------+---------+--------------+-------------------+-----------+--------------------------+---------------------------+---------------------------------+-------------------------------+-----------------------------+-----------------------------------+---------------------------------+-----------------------------+-----------------------------------+---------------------------------+-----------------------------+-----------------------------------+----------------------------------+---------------------------------+--------------------------------+--------------------------------+-------------------------------+--------------------------+-------------------------+-----------------+------------------------------+--------------------+----------------------------+-----------------------+----------------------------+

**Merging the One-hot encoded style columns**
- for instance, these columns contain a Y for one of the joint columns and empty for the rest hence merge those into 1 column with the name of the speficity

In [12]:
# The merging/combining columns
combine_and_drop(merged_df, 'CP Contact Area', [
    'CP Contact Area - Rear Left', 'CP Contact Area - Left', 'CP Contact Area - Front Left',
    'CP Contact Area - Rear', 'CP Contact Area - Top', 'CP Contact Area - Front',
    'CP Contact Area - Rear Right', 'CP Contact Area - Right', 'CP Contact Area - Front Right',
    'CP Contact Area - Bottom' 
])

combine_and_drop(merged_df, 'ADAS/ADS System Version', [
    'ADAS/ADS System Version', 'ADAS/ADS System Version - Unk', 'ADAS/ADS System Version CBI'
])

combine_and_drop(merged_df, 'ADAS/ADS Hardware Version', [
    'ADAS/ADS Hardware Version', 'ADAS/ADS Hardware Version - Unk', 'ADAS/ADS Hardware Version CBI'
])

combine_and_drop(merged_df, 'ADAS/ADS Software Version', [
    'ADAS/ADS Software Version', 'ADAS/ADS Software Version - Unk', 'ADAS/ADS Software Version CBI'
])

combine_and_drop(merged_df, 'Other Reporting Entities', [
    'Other Reporting Entities?', 'Other Reporting Entities? - Unk', 'Other Reporting Entities? - NA'
])

combine_and_drop(merged_df, 'Federal Regulatory Exemption', [
    'Federal Regulatory Exemption?', 'Other Federal Reg. Exemption',
    'Federal Reg. Exemption - Unk', 'Federal Reg. Exemption - No'
])

combine_and_drop(merged_df, 'State or Local Permit', [
    'State or Local Permit?', 'State or Local Permit'
])

combine_and_drop(merged_df, 'Source', [
    'Source - Complaint/Claim', 'Source - Telematics', 'Source - Law Enforcement',
    'Source - Field Report', 'Source - Testing', 'Source - Media',
    'Source - Other', 'Source - Other Text'
])

combine_and_drop(merged_df, 'Weather', [
    'Weather - Clear', 'Weather - Snow', 'Weather - Cloudy', 'Weather - Fog/Smoke',
    'Weather - Rain', 'Weather - Severe Wind', 'Weather - Other', 'Weather - Other Text'
])

combine_and_drop(merged_df, 'SV Contact Area', [
    'SV Contact Area - Rear Left', 'SV Contact Area - Left', 'SV Contact Area - Front Left',
    'SV Contact Area - Rear', 'SV Contact Area - Top', 'SV Contact Area - Front',
    'SV Contact Area - Rear Right', 'SV Contact Area - Right', 'SV Contact Area - Front Right',
    'SV Contact Area - Bottom'
])

combine_and_drop(merged_df, 'Data Availability', [
    'Data Availability - EDR', 'Data Availability - Police Rpt', 'Data Availability - Telematics',
    'Data Availability - Complaints', 'Data Availability - Video', 'Data Availability - Other',
    'Data Availability - No Data'
])

# drop all the unknown columns as they can be empty and imputed later (22 columns)
merged_df = merged_df.loc[:, ~merged_df.columns.str.contains('Unknown')]
print(tabulate(merged_df.head(), headers='keys', tablefmt='grid'))
print(f"Number of columns after combining connected columns': {merged_df.shape[1]}")

+----+-------------+------------------+--------------------+---------------+----------------+---------------+--------------------------+-------------+-----------------+---------------+---------+--------------+-------------------+-----------+--------------------------------+-----------------+------------------------------+--------------------+-----------------+------------------------+-------------------------+--------------------+---------------------------------------------------+---------------------------------------------------+---------------------------------------------------+--------------------+---------+---------------------------------------------------+-------------------+-------------------+-----------------------+----------------------------+--------------------+--------------------+-----------------------------------+--------------------+-------------------------+-----------------------------+-------------------------+-------------------------+---------------------------

**Drop the columns not needed**

In [13]:
cols_to_drop = [
    'Report Type', 'Report Month', 'Report Year', 'Report Submission Date','Driver / Operator Type'
    'Notice Received Date', 'Notice Received Date','Reporting Entity', 'Operating Entity', 
    'Serial Number','Data Availability',
    'Latitude', 'Longitude', 'Address', 'Zip Code', 
    'Investigating Agency', 'Rep Ent Or Mfr Investigating?', 'Investigating Officer Name',
    'Investigating Officer Phone', 'Investigating Officer Email',
    'Other Reporting Entities', 'Federal Regulatory Exemption',
    'Within ODD? - CBI','Within ODD?',
    'Same Incident ID', 'Same Vehicle ID',
    'Narrative', 'Narrative - CBI?',
    'Report Type', 'Report Month', 'Report Year',
    'VIN', 
    'Law Enforcement Investigating?','Source'
]

# Drop the unwanted columns
cols_to_drop = [col for col in cols_to_drop if col in merged_df.columns]
merged_df.drop(columns=cols_to_drop, inplace=True)
print(tabulate(merged_df.head(8), headers='keys', tablefmt='grid'))
print(f"Number of columns after combining connected columns': {merged_df.shape[1]}")

+----+-------------+------------------+---------------+-------------+--------------+-----------+--------------------------------+-----------------+------------------------------+-----------------+-------------------------+--------------------+---------+-------------------+-------------------+-----------------------+----------------------------+--------------------+-------------------------+-----------------------------------+--------------------+-------------------------+-----------------------------+-------------------------+-------------------------+-----------------------------+-------------------------+----------------------------------+---------------------------+-----------------------------------+-----------+--------------------------------------------------------------------+
|    | Report ID   |   Report Version | Make          | Model       |   Model Year |   Mileage | Driver / Operator Type         | ADS Equipped?   | Automation System Engaged?   | Incident Date   | Incident

## **<span style="color:black;">Section 3: Assessing Missing Values</span>**
- Balancing act of reducing columns to see how much cell informaation can be saved
- reduce any rows that may not be in the UK STATS19 rows 
- reduce any rows that are more than 60% missing 
- see the number of missing/nan values across the columns

In [14]:
# Function to count and print the overall sum of missing, empty, NaN, and "unknown" values for each column
def count_and_sum_missing_values(df):
    """
    Purpose: Count and sum missing, empty, NaN, and 'unknown' values for each column in the DataFrame.
    Input: DataFrame
    Output: Prints a summary table with the counts and percentages of missing values.
    """

    # Initialize a list to store the summary for each column
    overall_summary = []
    for col in df.columns:
        nan_count = df[col].apply(lambda x: pd.isna(x)).sum()  # Count NaN values explicitly
        unknown_count = df[col].apply(lambda x: str(x).strip().lower() == 'unknown').sum()  # Count 'unknown' values
        total =  nan_count + unknown_count
        percentage = (total / len(df)) * 100  # Calculate percentage
        overall_summary.append((col, nan_count, unknown_count, total, percentage))

    # Create a DataFrame for the summary and sort by the highest percentage
    summary_df = pd.DataFrame(overall_summary, columns=['Column', 'NaN', 'Unknown', 'Total', 'Percentage'])
    summary_df = summary_df.sort_values(by='Percentage', ascending=False)
    print(tabulate(summary_df, headers='keys', tablefmt='grid')) # make table to clearly see the results

# Call the function on the dataframe
count_and_sum_missing_values(merged_df)

+----+---------------------------------+-------+-----------+---------+--------------+
|    | Column                          |   NaN |   Unknown |   Total |   Percentage |
+====+=================================+=======+===========+=========+==============+
| 23 | CP Was Vehicle Towed?           |   832 |      1904 |    2736 |   62.5801    |
+----+---------------------------------+-------+-----------+---------+--------------+
| 22 | CP Any Air Bags Deployed?       |   832 |      1773 |    2605 |   59.5837    |
+----+---------------------------------+-------+-----------+---------+--------------+
| 27 | SV Were All Passengers Belted?  |     0 |      2325 |    2325 |   53.1793    |
+----+---------------------------------+-------+-----------+---------+--------------+
| 19 | Highest Injury Severity Alleged |     0 |      2147 |    2147 |   49.108     |
+----+---------------------------------+-------+-----------+---------+--------------+
| 26 | SV Was Vehicle Towed?           |     0 |      

**Dropping Columns with more than 45% missing values and unneeded columns**

In [15]:
# Columns to drop with more than 50% missing values
columns_to_remove = ['CP Pre-Crash Movement', 'CP Any Air Bags Deployed?', 'CP Was Vehicle Towed?', 'SV Any Air Bags Deployed?',
                     'SV Was Vehicle Towed?', 'SV Were All Passengers Belted?', 'Driver / Operator Type']

# Drop the columns
merged_df.drop(columns=columns_to_remove, inplace=True)

# Replace any value containing "Unknown" (case-insensitive) with NaN
merged_df = merged_df.applymap(
    lambda x: np.nan if isinstance(x, str) and 'unknown' in x.lower() else x
)

count_and_sum_missing_values(merged_df)

+----+---------------------------------+-------+-----------+---------+--------------+
|    | Column                          |   NaN |   Unknown |   Total |   Percentage |
+====+=================================+=======+===========+=========+==============+
| 18 | Highest Injury Severity Alleged |  2147 |         0 |    2147 |   49.108     |
+----+---------------------------------+-------+-----------+---------+--------------+
| 14 | Roadway Description             |  1086 |         0 |    1086 |   24.8399    |
+----+---------------------------------+-------+-----------+---------+--------------+
| 17 | Crash With                      |   922 |         0 |     922 |   21.0887    |
+----+---------------------------------+-------+-----------+---------+--------------+
| 16 | Lighting                        |   858 |         0 |     858 |   19.6249    |
+----+---------------------------------+-------+-----------+---------+--------------+
| 13 | Roadway Surface                 |   851 |      

/var/folders/0k/whv99z5n5rnb1xp8n07krpcw0000gn/T/ipykernel_20376/4195408928.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  merged_df = merged_df.applymap(


## **<span style="color:black;">Section 4: Fixing Formats and Standardisng Values to match with STATS19</span>**


**Converting to Datetime Datatypes**

In [16]:
def clean_incident_date(val):
    """
    Purpose: Clean and parse incident date strings into datetime objects.
    Input: String representing the incident date
    Output: Parsed datetime object or NaT if parsing fails
    """
    try:
        val = str(val).strip()
        if val.lower() == '':
            return pd.NaT
        # Try parsing common formats
        return parser.parse(val, dayfirst=False, yearfirst=False)
    except:
        return pd.NaT

merged_df['Incident Date'] = merged_df['Incident Date'].apply(clean_incident_date)

# Convert to datetime (assumes format like 'NOV-2021')
merged_df['Incident Date'] = pd.to_datetime(
    merged_df['Incident Date'], format='%b-%Y', errors='coerce'
)
# Convert time strings to time objects with hours and minutes
merged_df['Incident Time (24:00)'] = pd.to_datetime(
    merged_df['Incident Time (24:00)'], format='%H:%M', errors='coerce' # nat if coerece fails
).dt.time


# Drop rows where 'Incident Date' is NaT as harder to identify as missing
print("Remaining NaT values:", merged_df['Incident Date'].isna().sum())
merged_df = merged_df.dropna(subset=['Incident Date'])
print(f"Shape after dropping NaT values in 'Incident Date': {merged_df.shape}")

Remaining NaT values: 19
Shape after dropping NaT values in 'Incident Date': (4353, 25)


**Converting Numerical Datetime Columns**

In [17]:
# Numerical columns to convert to numeric types
numerical_columns = ['Report Version', 'Mileage', 'Posted Speed Limit (MPH)', 'SV Precrash Speed (MPH)','Model Year']
merged_df[numerical_columns] = merged_df[numerical_columns].apply(pd.to_numeric, errors='coerce')
print(tabulate(merged_df.head(10), headers='keys', tablefmt='grid'))

+----+-------------+------------------+---------------+--------------+--------------+-----------+-----------------+------------------------------+---------------------+-------------------------+--------------------+---------+-------------------+-------------------+-----------------------+----------------------------+--------------------+-------------------------+-----------------------------------+--------------------+-------------------------+---------------------------+-----------------------------------+-----------+--------------------------------------------------------------------+
|    | Report ID   |   Report Version | Make          | Model        |   Model Year |   Mileage | ADS Equipped?   | Automation System Engaged?   | Incident Date       | Incident Time (24:00)   | City               | State   | Roadway Type      | Roadway Surface   | Roadway Description   |   Posted Speed Limit (MPH) | Lighting           | Crash With              | Highest Injury Severity Alleged   | Prop

### **Matching the STATS19 Columns name**
**STATS 19 MATCHING Columns**
* Report ID -> accident_index 
* Report Version -> accident_reference 
* Make -> make (from generic_make_model)
* Model -> model (from generic_make_model)
* Model Year -> Model Year (from age_of_vehicle)
* Incident Date -> date
* Incident Time (24:00) -> time
* city -> city (from longitude and latitude)
* state -> county
* Roadway Type -> road_type
* Roadway Surface -> road_surface_conditions
* Posted Speed Limit (MPH) -> speed_limit
* Lighting -> light_conditions
* Crash With -> hit_object_in_carriageway and hit_object_off_carriageway 
* Highest Injury Severity Alleged -> accident_severity
* SV Precrash Movement -> vehicle_manoeuvre
* SV Precrash Speed (MPH) 
* Weather -> weather_conditions
* SV Contact Area -> first_point_of_impact

**UNMATCHED**
* SV Any Air Bags Deployed?  
* CP Contact Area 
* Property Damage 
* Mileage 
* ADS Equipped 
* Automatic System Engaged? 
* Roadway Description

**ADD**
- Country (US)

In [18]:
# Add a new column 'Country' and fill it with the value 'US' for all rows
merged_df['Country'] = 'US'
columns_to_drop_us_only = [
    'Roadway Description', 
    'Mileage', 
    'Property Damage?', 
    'SV Precrash Speed (MPH)', 
    'CP Contact Area'
]

merged_df.drop(columns=columns_to_drop_us_only, inplace=True)
print(f"Shape of the dataframe after dropping US-only columns: {merged_df.shape}")
print(tabulate(merged_df.head(10), headers='keys', tablefmt='grid'))

Shape of the dataframe after dropping US-only columns: (4353, 21)
+----+-------------+------------------+---------------+--------------+--------------+-----------------+------------------------------+---------------------+-------------------------+--------------------+---------+-------------------+-------------------+----------------------------+--------------------+-------------------------+-----------------------------------+-------------------------+-----------+--------------------------------------------------------------------+-----------+
|    | Report ID   |   Report Version | Make          | Model        |   Model Year | ADS Equipped?   | Automation System Engaged?   | Incident Date       | Incident Time (24:00)   | City               | State   | Roadway Type      | Roadway Surface   |   Posted Speed Limit (MPH) | Lighting           | Crash With              | Highest Injury Severity Alleged   | SV Pre-Crash Movement   | Weather   | SV Contact Area                              

**Renaming and Mapping values to match value names from STATS19**
- Easy comparsion when similar names and when putting into ML models

In [19]:
# Function to rename values within a cell
def remap_contact_area(cell):
    """
    Purpose: Rename values in a column based on a mapping dictionary.
    Input:
        df: DataFrame to modify
        column: Name of the column to rename values
        mapping: Dictionary with old values as keys and new values as values
    Output: DataFrame with renamed values in the specified column
    """
    if pd.isna(cell):
        return cell
    values = [v.strip() for v in cell.split(',')]
    mapped = [value_mapping.get(v, v) for v in values]
    return ', '.join(sorted(set(mapped)))  # Remove duplicates and sort for consistency

# Define the value mapping for SV Contact Area
value_mapping = {
    'Rear Left': 'Back',
    'Rear': 'Back',
    'Rear Right': 'Back',
    'Left': 'Nearside',
    'Top': 'Top',
    'Front': 'Front',
    'Front Right': 'Front',
    'Front Left': 'Front',
    'Right': 'Offside',
    'Bottom': 'Bottom'
}

# Apply the transformation
merged_df['SV Contact Area'] = merged_df['SV Contact Area'].apply(remap_contact_area)

# Preview
merged_df.head(10)

,Report ID,Report Version,Make,Model,Model Year,ADS Equipped?,Automation System Engaged?,Incident Date,Incident Time (24:00),City,...,Roadway Type,Roadway Surface,Posted Speed Limit (MPH),Lighting,Crash With,Highest Injury Severity Alleged,SV Pre-Crash Movement,Weather,SV Contact Area,Country
0,10003-2800,4,International,LT,2022.0,No,NaN,2025-11-21,19:57:00,Maxatawny Tounship,...,Street,Dry,25.0,Dark - Not Lighted,Other Fixed Object,NaN,Proceeding Straight,,Front,US
1,10003-2932,1,International,LT,2022.0,Yes,NaN,2021-11-08,19:57:00,Maxatawny Township,...,Street,Dry,25.0,Dark - Not Lighted,Other Fixed Object,NaN,Proceeding Straight,,Front,US
2,10003-3210,1,International,LT625,2021.0,Yes,NaN,2022-04-08,15:52:00,Tucson,...,Highway / Freeway,Dry,65.0,Daylight,Other Fixed Object,No Injuries Reported,Proceeding Straight,Clear,Front,US
3,10003-4179,1,International,LT,2022.0,No,ADAS,2025-10-22,15:09:00,NaN,...,Highway / Freeway,Dry,65.0,Daylight,Passenger Car,NaN,Proceeding Straight,Clear,Front,US
4,10003-6050,2,International,LT625,2022.0,No,NaN,2025-12-22,08:20:00,Natural Bridge,...,Highway / Freeway,Dry,70.0,Daylight,Other Fixed Object,No Injuries Reported,Proceeding Straight,Clear,"Back, Front, Nearside, Offside",US
5,10039-10291,3,BMW,i5 xDrive40,NaN,No,ADAS,2025-03-25,16:15:00,Washington,...,Highway / Freeway,Dry,NaN,Daylight,Other Fixed Object,Minor,Changing Lanes,Clear,Front,US
6,10039-10316,4,BMW,XM,2023.0,No,ADAS,2025-03-25,14:06:00,Benton County,...,Highway / Freeway,NaN,65.0,NaN,First Responder Vehicle,Minor,NaN,,"Back, Front",US
7,10039-1520,4,BMW,X5M,2021.0,No,ADAS,2025-08-21,NaT,sacramento,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,US
9,10039-3261,4,BMW,540i,2022.0,No,NaN,2025-06-22,13:27:00,Watermill,...,Highway / Freeway,NaN,NaN,NaN,Heavy Truck,Minor,Proceeding Straight,,Front,US
10,10039-3997,3,BMW,X5 sDrive40i,2020.0,No,ADAS,2025-05-22,22:25:00,Refugio,...,Highway / Freeway,NaN,75.0,NaN,Pole / Tree,Serious,Proceeding Straight,Clear,Front,US


## **<span style="color:black;">Section 5: Save Cleaned Dataframe</span>**

In [20]:
print("Shape of merged_df:", merged_df.shape)
merged_df.to_csv('/Users/mahnooriqbal/COMP702 Project/ML-AI-Risk-Analysis-AV-Data-/Datasets/US-cleaned_data.csv', index=False)

Shape of merged_df: (4353, 21)
